# requirements

In [1]:
import pandas as pd
import numpy as np
import requests_html
from requests_html import HTMLSession
import os
import regex as re
import requests
from xpinyin import Pinyin
from time import sleep

In [2]:
session = HTMLSession()
pinyin = Pinyin()

In [3]:
headers = {
    'user-agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:54.0) Gecko/20100101 Firefox/54.0',
    'accept': 'text/html, application/xhtml+xml, application/xml; q=0.9, image/webp, */*; q=0.8',
    'referer': 'http://www.gufengmh.com/manhua/'
}

In [4]:
# image names
pattern_chapterImagesList = re.compile('var chapterImages = \["(.+?)"\]')
# chapter url
pattern_chapterPath = re.compile('var chapterPath = "(.+?)"')
# chapter title
pattern_title = re.compile('var pageTitle = "(.+?)"')

## get the catalogs
+ **chapter_urls** (所有章节的url)

In [5]:
def get_the_chapter_urls(comic_name):
    main_url = 'http://www.gufengmh.com/manhua/'
    pattern_catalog = re.compile('/manhua/' + comic_name + '/\d+.html')
    catalog_url = os.path.join(main_url, comic_name)
    res = session.get(catalog_url)
    chapter_urls = []
    for url in res.html.links:
        if pattern_catalog.match(url):
            chapter_urls.append('http://www.gufengmh.com'+url)
    return chapter_urls

## get the first chapter

In [6]:
def get_chapter_details(chapter_url):
    res = session.get(chapter_url)

    # get the chapterPath
    temp_chapterPath = pattern_chapterPath.search(res.text)
    chapterPath = temp_chapterPath.group(1)
    print(chapterPath)

    # get the chapterImages urls
    temp_chapterImagesList = pattern_chapterImagesList.search(res.text)
    chapterImagesList_abbr = temp_chapterImagesList.group(1).split('","')
    chapterImagesList = []
    for i in chapterImagesList_abbr:
        chapterImagesList.append('http://res.gufengmh.com/'+chapterPath+i)

    # get the chapterTitle
    temp_chapterTitle = pattern_title.search(res.html.full_text)
    chapterTitle = temp_chapterTitle.group(1)
    chapterTitle = chapterTitle.replace('在线观看', '')
    
    return chapterImagesList, chapterTitle

In [7]:
def download(img_url, img_name):
    sleep(1)
    img = session.get(img_url, headers=headers)
    with open(img_name+'.jpg', 'ab') as f:
        f.write(img.content)

In [9]:
def download_chapter(chapterImagesList, file_path):
    sleep(10)
    if not os.path.isdir(file_path):
        os.mkdir(file_path)
    for index, img_url in enumerate(chapterImagesList):
        print(img_url)
        download(img_url, os.path.join(file_path, str(index)))

## main

In [10]:
def main(comic_name_chn, comic_file_path):
    comic_file_path = os.path.join(comic_file_path, comic_name_chn)
    if not os.path.isdir(comic_file_path):
        os.mkdir(comic_file_path)
    comic_name = pinyin.get_pinyin(comic_name_chn, '')
    chapter_urls = get_the_chapter_urls(comic_name)
    for chapter_url in chapter_urls:
        chapterImagesList, chapterTitle = get_chapter_details(chapter_url)
        chapter_file_path = os.path.join(comic_file_path, chapterTitle)
        download_chapter(chapterImagesList, chapter_file_path)

In [12]:
main('爆炸头武士', 'D:\\zhangyudong\\Netease-US\\comic_spider')

# testing
+ 存在一个问题, 爬下来的漫画部分chapter的图片显示"格式不支持"

In [13]:
main('爆炸头武士', 'D:\\zhangyudong\\Netease-US\\comic_spider')